# Spark Structured Streaming Example

Purpose: Reads a stream of messages from a Kafka topic and writes the running totals to json file.

References: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html

Author:  Gary A. Stafford

Date: 2022-12-16

Updated by Vasilios Anagnostopoulos 30/09/2025

In [33]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructField,
    StructType,
    IntegerType,
    StringType,
    FloatType,
    TimestampType,
    BooleanType,
)

In [34]:
WINDOW_WATERMARK = "10 seconds"
WINDOW_DURATION = "20 seconds"
WINDOW_SLIDE = "10 seconds"
PROCESSING_TIME = "20 seconds"
SHOW_REFRESH = 20

BOOTSTRAP_SERVERS = "kafka:29092"
TOPIC_PURCHASES = "demo.purchases"

KAFKA_OPTIONS = {
    "kafka.bootstrap.servers": BOOTSTRAP_SERVERS,
    "subscribe": TOPIC_PURCHASES,
    "startingOffsets": "earliest",
}

In [35]:
spark = SparkSession.builder.remote("sc://127.0.0.1:15002").appName("kafka-streaming-query").getOrCreate()
df_sales = spark.readStream.format("kafka").options(**KAFKA_OPTIONS).load()

In [36]:
schema = StructType(
    [
        StructField("transaction_time", TimestampType(), False),
        StructField("transaction_id", StringType(), False),
        StructField("product_id", StringType(), False),
        StructField("price", FloatType(), False),
        StructField("quantity", IntegerType(), False),
        StructField("is_member", BooleanType(), True),
        StructField("member_discount", FloatType(), True),
        StructField("add_supplements", BooleanType(), True),
        StructField("supplement_price", FloatType(), True),
    ]
)

ds_sales = (
    df_sales.selectExpr("CAST(value AS STRING)")
    .select(F.from_json("value", schema=schema).alias("data"))
    .select("data.*")
    .withColumn("sales", F.col("price")*F.col("quantity"))
    .where(F.col('product_id') == 'SC04')
    .groupBy("product_id")
    .agg(F.sum("sales"), F.sum("quantity"), F.max("transaction_time"))
    #.orderBy(F.col("product_id").desc())
    .select(
        "product_id",
        F.col('max(transaction_time)'),
        F.format_number("sum(sales)", 2).alias("total_sales"),
        F.format_number("sum(quantity)", 0).alias("total_items"),
        F.format_number(F.col("sum(sales)")/F.col("sum(quantity)"), 2).alias("mean_price"),
    )
    .coalesce(1)
    .writeStream.queryName("streaming_to_console")
    .trigger(processingTime=PROCESSING_TIME)
    #.outputMode("complete")
    .outputMode("update")
    .format("memory")
    .start()
)

In [37]:
# data display auto-refresh reference: https://stackoverflow.com/a/61922073/580268

from IPython.display import display, clear_output
from time import sleep

while True:
    clear_output(wait=True)
    display(ds_sales.status)
    display(spark.sql("SELECT * FROM streaming_to_console").show())
    sleep(SHOW_REFRESH)  # refresh every SHOW_REFRESH seconds

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

+----------+---------------------+-----------+-----------+----------+
|product_id|max(transaction_time)|total_sales|total_items|mean_price|
+----------+---------------------+-----------+-----------+----------+
|      SC04| 2025-10-03 16:57:...|      17.97|          3|      5.99|
|      SC04| 2025-10-03 16:59:...|      23.96|          4|      5.99|
|      SC04| 2025-10-03 17:00:...|      35.94|          6|      5.99|
|      SC04| 2025-10-03 17:02:...|      47.92|          8|      5.99|
+----------+---------------------+-----------+-----------+----------+



None

KeyboardInterrupt: 

In [38]:
ds_sales.stop()